In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir

In [2]:
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Conv1D, Input, Concatenate, GlobalMaxPooling1D
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras import optimizers
from keras.regularizers import l1,l2,l1_l2

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [3]:
df_train = pd.read_csv('df_train.csv')

In [4]:
df_train = df_train[df_train['target'] > 8]

In [5]:
df_med = df_train.drop(df_train.iloc[:, :36], axis = 1) 
df_med.drop(df_med.iloc[:, 72:108], inplace = True, axis = 1) 

In [6]:
df_small = df_train.drop(df_train.iloc[:, :54], axis = 1) 
df_small.drop(df_small.iloc[:, 36:90], inplace = True, axis = 1) 

In [7]:
df_smaller = df_train.drop(df_train.iloc[:, :64], axis = 1) 
df_smaller.drop(df_smaller.iloc[:, 18:80], inplace = True, axis = 1) 

In [8]:
df_smooth = df_train.T
df_med = df_med.T
df_small = df_small.T
df_smaller = df_smaller.T

In [9]:
sequences_smooth = list()
for i in range(df_smooth.shape[1]):
    values = df_smooth.iloc[:-1,i].values
    sequences_smooth.append(values)
targets_smooth = df_smooth.iloc[-1, :].values

In [10]:
sequences_med = list()
for i in range(df_med.shape[1]):
    values = df_med.iloc[:-1,i].values
    sequences_med.append(values)
targets_med = df_med.iloc[-1, :].values

In [11]:
sequences_small = list()
for i in range(df_small.shape[1]):
    values = df_small.iloc[:-1,i].values
    sequences_small.append(values)
targets_small = df_small.iloc[-1, :].values

sequences_smaller = list()
for i in range(df_smaller.shape[1]):
    values = df_smaller.iloc[:-1,i].values
    sequences_smaller.append(values)
targets_smaller = df_smaller.iloc[-1, :].values

In [12]:
len(sequences_smooth), len(sequences_med), len(sequences_small)

(7605, 7605, 7605)

In [13]:
targets = targets_smooth

In [14]:
sequences_smooth, targets_smooth

([array([12.33246871, 14.10736319, 15.51341642, 14.45530238, 14.3204542 ,
         16.98976183, 16.90538619, 15.28010431, 15.96696703, 14.99131704,
         16.00158291, 14.90505853, 14.04414502, 13.23188736, 14.31276487,
         14.36089006, 13.51444411, 13.37715899, 15.29147947, 16.06817465,
         16.71313435, 16.82797351, 16.78007106, 17.47849653, 18.87704447,
         18.78897083, 18.75673732, 19.80283023, 18.31834486, 16.34776212,
         17.31794091, 17.17892645, 16.72184341, 14.53457812, 13.69416156,
         15.32770855, 14.90159189, 15.34498442, 14.51592579, 15.43997192,
         15.2586018 , 15.62478912, 17.31759408, 18.10352302, 18.51577681,
         17.73742931, 18.67597742, 20.16079401, 17.95987033, 19.30431259,
         20.27269593, 20.02521552, 17.63329393, 19.49734267, 18.62990474,
         16.88511273, 16.37370052, 17.43799934, 15.89522014, 14.33765162,
         13.59721653, 12.95150795, 13.41796573, 11.80866453, 11.78445046,
         12.17147108, 12.0574207 , 12.

In [15]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.utils import np_utils

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(targets)
encoded_y = encoder.transform(targets)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_y)
dummy_y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [16]:
targets = dummy_y

In [17]:
from sklearn.model_selection import train_test_split
X_train_smooth, X_train_med, X_train_small, X_train_smaller, y_train = sequences_smooth, sequences_med, sequences_small, sequences_smaller, targets
X_train_smooth, X_val_smooth, X_train_med, X_val_med, X_train_small, X_val_small, X_train_smaller, X_val_smaller, y_train, y_val = train_test_split(X_train_smooth, X_train_med, X_train_small, X_train_smaller, y_train, test_size=0.08, random_state=1)

In [18]:
len(X_train_smooth), len(X_train_med), len(X_train_small)

(6996, 6996, 6996)

In [19]:
len(y_train), len(y_val)

(6996, 609)

In [20]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.externals.joblib import dump, load
sc1 = load('std_scaler_smooth.bin')
X_train_smooth = sc1.transform(X_train_smooth)
# X_test_smooth = sc1.transform(X_test_smooth)
X_val_smooth = sc1.transform(X_val_smooth)

sc2 = load('std_scaler_med.bin')
X_train_med = sc2.transform(X_train_med)
# X_test_med = sc2.transform(X_test_med)
X_val_med = sc2.transform(X_val_med)

sc3 = load('std_scaler_small.bin')
X_train_small = sc3.transform(X_train_small)
# X_test_small = sc3.transform(X_test_small)
X_val_small = sc3.transform(X_val_small)

sc4 = load('std_scaler_smaller.bin')
X_train_smaller = sc4.transform(X_train_smaller)
# X_test_smaller = sc4.transform(X_test_smaller)
X_val_smaller = sc4.transform(X_val_smaller)

/home/hongyu/anaconda3/envs/keras/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [21]:
X_train_smooth.shape, X_val_smooth.shape

((6996, 144), (609, 144))

In [22]:
X_train_med.shape, X_val_med.shape

((6996, 72), (609, 72))

In [23]:
X_train_small.shape, X_val_small.shape

((6996, 36), (609, 36))

In [24]:
X_train_smaller.shape, X_val_smaller.shape

((6996, 18), (609, 18))

In [25]:
X_train_smooth = np.reshape(X_train_smooth, (X_train_smooth.shape[0], X_train_smooth.shape[1], 1))
# X_test_smooth = np.reshape(X_test_smooth, (X_test_smooth.shape[0], X_test_smooth.shape[1], 1))
X_val_smooth = np.reshape(X_val_smooth, (X_val_smooth.shape[0], X_val_smooth.shape[1], 1))

X_train_med = np.reshape(X_train_med, (X_train_med.shape[0], X_train_med.shape[1], 1))
# X_test_med = np.reshape(X_test_med, (X_test_med.shape[0], X_test_med.shape[1], 1))
X_val_med = np.reshape(X_val_med, (X_val_med.shape[0], X_val_med.shape[1], 1))

X_train_small = np.reshape(X_train_small, (X_train_small.shape[0], X_train_small.shape[1], 1))
# X_test_small = np.reshape(X_test_small, (X_test_small.shape[0], X_test_small.shape[1], 1))
X_val_small = np.reshape(X_val_small, (X_val_small.shape[0], X_val_small.shape[1], 1))

X_train_smaller = np.reshape(X_train_smaller, (X_train_smaller.shape[0], X_train_smaller.shape[1], 1))
# X_test_smaller = np.reshape(X_test_smaller, (X_test_smaller.shape[0], X_test_smaller.shape[1], 1))
X_val_smaller = np.reshape(X_val_smaller, (X_val_smaller.shape[0], X_val_smaller.shape[1], 1))

In [26]:
X_train_smooth.shape, X_val_smooth.shape

((6996, 144, 1), (609, 144, 1))

In [27]:
X_train_med.shape, X_val_med.shape

((6996, 72, 1), (609, 72, 1))

In [28]:
X_train_small.shape, X_val_small.shape

((6996, 36, 1), (609, 36, 1))

In [29]:
X_train_smaller.shape, X_val_smaller.shape

((6996, 18, 1), (609, 18, 1))

In [31]:
#this base model is one branch of the main model
#it takes a time series as an input, performs 1-D convolution, and returns it as an output ready for concatenation
def get_base_model(input_len, nodes):
    #the input is a time series of length n and width 19
    input_seq = Input(shape=(input_len, 1))
    lstmed = LSTM(nodes, return_sequences=True, kernel_regularizer=l2(0.01))(input_seq)
    drop = Dropout(0.3)(lstmed)
    processed = Flatten()(drop)
    model = Model(inputs=input_seq, outputs=processed)
    return model

#this is the main model
#it takes the original time series and its down-sampled versions as an input, and returns the result of classification as an output
def main_model(inputs_lens = [18, 36, 72, 144], nodes = [18, 36, 72, 144]):
    #the inputs to the branches are the original time series, and its down-sampled versions
    input_smallseq = Input(shape=(inputs_lens[1], 1))
    input_medseq = Input(shape=(inputs_lens[2] , 1))
    input_smallerseq = Input(shape=(inputs_lens[0], 1))
    input_smoothseq = Input(shape=(inputs_lens[3], 1))
    
    #the more down-sampled the time series, the shorter the corresponding filter
    base_net_small = get_base_model(inputs_lens[1], nodes[1])
    base_net_med = get_base_model(inputs_lens[2], nodes[2])
    base_net_smaller = get_base_model(inputs_lens[0], nodes[0])
    base_net_smooth = get_base_model(inputs_lens[3], nodes[3])
    embedding_small = base_net_small(input_smallseq)
    embedding_med = base_net_med(input_medseq)
    embedding_smaller = base_net_smaller(input_smallerseq)
    embedding_smooth = base_net_smooth(input_smoothseq)
    
    #concatenate all the outputs
    merged = Concatenate()([embedding_small, embedding_med, embedding_smaller, embedding_smooth])
    out = Dense(9, activation='softmax')(merged)
    model = Model(inputs=[input_smallseq, input_medseq, input_smallerseq, input_smoothseq], outputs=out)
    return model

In [32]:
model = main_model()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [33]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 36, 1)        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 72, 1)        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 18, 1)        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 144, 1)       0                                            
__________________________________________________________________________________________________
model_1 (M

In [33]:
import os
model_filename = 'model-{epoch:03d}-{acc:03f}-{val_acc:03f}-{val_loss:03f}.h5'
checkpoint_path = os.path.join('temp_btn/', model_filename)
monitor = EarlyStopping(monitor = 'val_loss', min_delta = 1e-3, patience = 10, verbose = 1, mode = 'auto')
checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  
adam = optimizers.Adam(lr=0.000001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [34]:
# Fitting our model [input_smallseq, input_medseq, input_origseq, input_smoothseq]
print('Train...')
history = model.fit([X_train_small, X_train_med, X_train_smaller, X_train_smooth], y_train, validation_data = ([X_val_small, X_val_med, X_val_smaller, X_val_smooth], y_val), callbacks = [monitor, checkpoint], verbose = 1, batch_size = 64, nb_epoch = 100)

Train...


/home/hongyu/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 6996 samples, validate on 609 samples
Epoch 1/100
6996/6996 [==============================] - 13s 2ms/step - loss: 0.0641 - acc: 0.9818 - val_loss: 0.1271 - val_acc: 0.9573

Epoch 00001: val_loss improved from inf to 0.12710, saving model to temp_btn/model-001-0.981847-0.957307-0.127104.h5
Epoch 2/100
6996/6996 [==============================] - 13s 2ms/step - loss: 0.0825 - acc: 0.9760 - val_loss: 0.1889 - val_acc: 0.9376

Epoch 00002: val_loss did not improve
Epoch 3/100
6996/6996 [==============================] - 13s 2ms/step - loss: 0.0765 - acc: 0.9806 - val_loss: 0.1580 - val_acc: 0.9475

Epoch 00003: val_loss did not improve
Epoch 4/100
6996/6996 [==============================] - 13s 2ms/step - loss: 0.0597 - acc: 0.9848 - val_loss: 0.1194 - val_acc: 0.9639

Epoch 00004: val_loss improved from 0.12710 to 0.11937, saving model to temp_btn/model-004-0.984848-0.963875-0.119370.h5
Epoch 5/100
6996/6996 [==============================] - 13s 2ms/step - loss: 0.0770 - acc:

In [ ]:
with open('/trainHistory', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

In [40]:
# serialize model to JSON
model_json = model.to_json()
with open("/home/hongyu/Documents/Spring2020/ECE_research/signal_analysis/data_18points/3_section_sliding5/MSLSTM_models/MSLSTM_buttomtonine_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/home/hongyu/Documents/Spring2020/ECE_research/signal_analysis/data_18points/3_section_sliding5/MSLSTM_weights/MSLSTM_buttomtonine_weight.h5")
print("Saved model to disk")

Saved model to disk


In [53]:
from keras.models import model_from_json
# load json and create model
json_file = open("/home/hongyu/Documents/Spring2020/ECE_research/signal_analysis/data_18points/3_section_sliding5/MSLSTM_models/MSLSTM_buttomtonine_model.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("/home/hongyu/Documents/Spring2020/ECE_research/signal_analysis/data_18points/3_section_sliding5/temp_btn/model-011-0.993711-0.973727.h5")
print("Loaded model from disk")

Loaded model from disk


In [30]:
from keras.models import model_from_json
# load json and create model
json_file = open("/home/hongyu/Documents/Spring2020/ECE_research/signal_analysis/data_18points/3_section_sliding5/MSLSTM_models/MSLSTM_buttomtonine_model.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("/home/hongyu/Documents/Spring2020/ECE_research/signal_analysis/data_18points/3_section_sliding5/MSLSTM_weights/model-001-0.979131-0.975369-btn.h5")
print("Loaded model from disk")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loaded model from disk
